<div class="frontmatter">
    <div style="display: flex; align-items: flex-start; justify-content: space-between;">
        <div>
             <h1 style="font-weight: bold;">Práctica 2: ESDA</h1>
            <h2>Autores: David Cantó, Roberto Rodero</h2>
            <h3>Máster en Tecnologías de la Información Geográfica, UAH</h3>
            <h3>Asignatura: Programación Avanzada</h3>
        </div>
</div>

# **Análisis de delitos y criminalidad en áreas urbanas**

## **1. Introducción**

### **Contexto**
La criminalidad en entornos urbanos es un fenómeno complejo influenciado por diversos factores socioeconómicos y espaciales. Comprender la distribución geográfica de los delitos es esencial para el diseño de estrategias de prevención y respuesta por parte de las autoridades. Este trabajo tiene como objetivo analizar los delitos cometidos en los distritos de la ciudad de Madrid, enfocándose en diversas categorías, como la seguridad ciudadana, el número de personas detenidas, los atestados y partes de accidentes, el consumo de alcohol en la vía pública, y las inspecciones y actuaciones en locales de espectáculos públicos y actividades recreativas, así como las multas de tráfico.

El análisis espacial permitirá identificar patrones y tendencias de criminalidad, contribuyendo a una mejor comprensión de estos fenómenos y proporcionando información útil para la toma de decisiones de seguridad y políticas públicas en este sentido.

## **Objetivos**
- Analizar la distribución geográfica de los delitos y las multas de tráfico en los distritos de Madrid y su evolución durante la última década.
- Evaluar la relación entre la densidad de población y la criminalidad en los distritos de Madrid.
- Generar cartografía y gráficos estadísticos que ayuden a comprender estos fenómenos.

## **Preguntas clave**
1. ¿Existe correlación espacial en los delitos de la Comunidad de Madrid?
2. ¿En el caso de que exista, de que tipo es?
3. ¿Cuáles son las áreas de Madrid con mayor número de multas de tráfico?
4. ¿Si agrupamos los distritos de Madrid según el número de delitos, adquieren una forma similar a las divisiones administrativas reales?

## **2. Conjunto de datos**

Para este estudio, se utilizarán datos públicos proporcionados por el Ayuntamiento de Madrid, a través del Portalde Datos Abiertos del Ayuntamiento de Madrid y se pueden consultar en el siguiente enlace:  [https://datos.madrid.es/sites/v/index.jsp?vgnextoid=bffff1d2a9fdb410VgnVCM2000000c205a0aRCRD&vgnextchannel=20d612b9ace9f310VgnVCM100000171f5a0aRCRD](https://datos.madrid.es/sites/v/index.jsp?vgnextoid=bffff1d2a9fdb410VgnVCM2000000c205a0aRCRD&vgnextchannel=20d612b9ace9f310VgnVCM100000171f5a0aRCRD). Los principales conjuntos de datos incluyen:

- Delitos relacionados con la seguridad ciudadana:
  - Relacionados con las personas
  - Relacionadas con el patrimonio
  - Por tenencia de armas
  - Por tenencia de drogas
  - Por consumo de drogas

- Personas detenidas e imputadas:

- Partes de accidente confeccionados y atestados:

- Consumo de alcohol en la vía pública:

- Intervenciones por protección a los consumidores y usuarios en vía pública:

- Inspecciones y actuaciones en locales de espectáculos públicos y actividades recreativas:

- Capa de distritos de la ciudad de Madrid

Se han utilizado datos de población por distritos, disponibles en el siguiente enlace: https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=0cccaebc07c1f710VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default.

Estos datos serán analizados en formato geoespacial (SHP, GeoJSON) y tabular (CSV, Excel), considerando su calidad, precisión y confiabilidad para garantizar resultados válidos.

Por su parte, el archivo ráster a analizar es un ráster de zonas verdes de la ciudad de Madrid, disponible en: https://geoportal.madrid.es/IDEAM_WBGEOPORTAL/dataset.iam?id=53896e7f-8b6b-4cbc-a0aa-bfbeee91563e.

En lo relativo a las estadísticas de las multas de tráfico en Madrid utilizadas para el análisis de patrones de puntos, se han obtenido en el siguiente enlace: https://datos.madrid.es/sites/v/index.jsp?vgnextoid=fb9a498a6bdb9410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD.

## **3. Exploración y procesamiento de datos**

En primer lugar se importaron los módulos y paquetes necesarios para poder explorar y procesar los conjuntos de datos con los que vamos a trabajar.

In [ ]:
!pip install rasterio
!pip install rasterstats
!pip install mapclassify
!pip install folium
!pip install mplcursors
!pip install pysal
!pip install libpysal
!pip install adjustText
!pip install contextily

In [ ]:
from google.colab import drive
from google.colab import files
import os
import pandas as pd
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import folium
import rasterio
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import plot_tree
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from rasterio.mask import mask
from rasterio.features import geometry_mask
from rasterstats import zonal_stats
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
from IPython.display import clear_output
import branca.colormap as cm
from sklearn.linear_model import LinearRegression
import warnings
from statsmodels.tsa.seasonal import seasonal_decompose
import dask.array as da
import mapclassify
from shapely.geometry import Point
import pysal
import contextily as cx
from pointpats import distance_statistics
from ipywidgets import interact, fixed
from libpysal.weights import Queen
from pysal.lib import weights
from splot.libpysal import plot_spatial_weights
import matplotlib.patches as mpatches
import io
from adjustText import adjust_text
from pysal.explore import esda
from splot.esda import (
    moran_scatterplot, lisa_cluster, plot_local_autocorrelation, plot_moran
)

Una vez cargados los paquetes necesarios, preparamos la carpeta de google drive desde la que vamos a trabajar. Dentro de ella se encuentran los conjuntos de datos que vamos a utilizar.

In [ ]:
# Conectar a Google Drive para enlazar las carpetas almacenadas en Drive con los archivos necesarios
drive.mount('/content/drive')
# Establecer la ruta de la carpeta principal
folder = '/content/drive/My Drive/P1_Avanzada'

# Listar las subcarpetas dentro de la carpeta principal
def listar_carpetas(carpeta):
    for name in os.listdir(carpeta):
        if os.path.isdir(os.path.join(carpeta, name)):
            print(name)

listar_carpetas(folder)

In [ ]:
# Establecer la ruta que contiene las carpetas con los datos de delitos, distritos y ráster de zonas verdes.
datos_2015 = os.path.join(folder, 'Delitos_2015')
datos_2016 = os.path.join(folder, 'Delitos_2016')
datos_2017 = os.path.join(folder, 'Delitos_2017')
datos_2018 = os.path.join(folder, 'Delitos_2018')
datos_2019 = os.path.join(folder, 'Delitos_2019')
datos_2020 = os.path.join(folder, 'Delitos_2020')
datos_2021 = os.path.join(folder, 'Delitos_2021')
datos_2022 = os.path.join(folder, 'Delitos_2022')
datos_2023 = os.path.join(folder, 'Delitos_2023')
datos_2024 = os.path.join(folder, 'Delitos_2024')
distritos_shp = os.path.join(folder, 'Distritos_shape', 'DISTRITOS.shp')
poblacion = os.path.join(folder, 'Poblacion', 'poblacion_1_enero.csv')
Zonas_verdes_rast = os.path.join(folder, 'Zonas_verdes_rast', 'Zonas_verdes_30m.tif')
multas = os.path.join(folder, 'Multas_5_2024', '202405detalle.csv')

Una vez localizadas y seleccionadas las carpetas que contiene los conjuntos de datos con los que vamos a trabajar, comenzamos a modificar estos conjuntos de datos. Para ello, el primer paso es seleccionar únicamente las hojas dentro de los archivos excel que nos interesan. Después, creamos para cada archivo excel una columna que incluira la fecha del archivo, extrayendola a partir del nombre del mismo. Por último, concatenamos las diferentes hojas y archivos para obtener una base de datos con la que poder trabajar.

In [ ]:
# Lista de las hojas a leer de los arcchivos excel
hojas_a_leer = ['SEGURIDAD', 'DETENIDOS X DISTRITOS', 'ACCIDENTES', 'VENTA AMBULANTE', 'CONSUMO ALCOHOL', 'LOCALES']

# Función para obtener los archivos Excel en una carpeta
def obtener_archivos_excel(carpeta):
    return [f for f in os.listdir(carpeta) if f.endswith('.xlsx') or f.endswith('.xls')]

# Obtener los archivos Excel de las carpetas datos_2015 y datos_2024
archivos_2015 = obtener_archivos_excel(datos_2015)
archivos_2016 = obtener_archivos_excel(datos_2016)
archivos_2017 = obtener_archivos_excel(datos_2017)
archivos_2018 = obtener_archivos_excel(datos_2018)
archivos_2019 = obtener_archivos_excel(datos_2019)
archivos_2020 = obtener_archivos_excel(datos_2020)
archivos_2021 = obtener_archivos_excel(datos_2021)
archivos_2022 = obtener_archivos_excel(datos_2022)
archivos_2023 = obtener_archivos_excel(datos_2023)
archivos_2024 = obtener_archivos_excel(datos_2024)

# Lista para almacenar los DataFrames de las hojas seleccionadas
todos_los_datos = []

# Función para procesar los archivos de una carpeta y concatenar las hojas seleccionadas por columnas
def procesar_archivos(carpeta, archivos):
    for archivo in archivos:
        # Leer todas las hojas del archivo Excel
        xls = pd.ExcelFile(os.path.join(carpeta, archivo))

        # Lista temporal para almacenar las hojas que se van a concatenar por columnas
        hojas_temporales = []

        # Obtener el nombre correcto del archivo (sin el sufijo)
        nombre_archivo_correcto = archivo.split('.')[0]

        # Variable para controlar si se está en la primera hoja
        es_primera_hoja = True

        # Iterar sobre las hojas del archivo y leer solo las que están en la lista `hojas_a_leer`
        for nombre_hoja in xls.sheet_names:
            if nombre_hoja in hojas_a_leer:
                # Leer la hoja en un DataFrame, omitiendo las dos primeras filas
                df = pd.read_excel(xls, sheet_name=nombre_hoja, skiprows=2)

                # Si es la primera hoja, añadir la columna 'Fecha' y 'Distritos' con el nombre del archivo
                if es_primera_hoja:
                    df['Fecha'] = nombre_archivo_correcto

                if not es_primera_hoja:
                    df = df.iloc[:, 1:]  # Eliminar la primera columna

                es_primera_hoja = False

                # Añadir el DataFrame de la hoja a la lista temporal
                hojas_temporales.append(df)

        # Si hay hojas seleccionadas para este archivo, concatenarlas por columnas
        if hojas_temporales:
            df_archivo = pd.concat(hojas_temporales, axis=1)
            # Añadir el DataFrame concatenado por columnas a la lista de todos los datos
            todos_los_datos.append(df_archivo)

# Procesar los archivos de las carpetas 2015 y 2024
procesar_archivos(datos_2015, archivos_2015)
procesar_archivos(datos_2016, archivos_2016)
procesar_archivos(datos_2017, archivos_2017)
procesar_archivos(datos_2018, archivos_2018)
procesar_archivos(datos_2019, archivos_2019)
procesar_archivos(datos_2020, archivos_2020)
procesar_archivos(datos_2021, archivos_2021)
procesar_archivos(datos_2022, archivos_2022)
procesar_archivos(datos_2023, archivos_2023)
procesar_archivos(datos_2024, archivos_2024)

# Concatenar todos los DataFrames de diferentes archivos Excel por filas
df_final = pd.concat(todos_los_datos, axis=0, ignore_index=True)

Después de esto, hacemos una primera exploración de los datos, para comprobar que se han unido correctamente las distintas hojas y excels.

In [ ]:
df_final.head()

In [ ]:
df_final.info()

Algunas variables contienen la misma información pero han recibido nombres diferentes. Por ello, lo primero que tenemos que hacer es concatenar los datos del mismo tipo.

In [ ]:
# Selecciona las columnas que quieres combinar
columnas_a_combinar = ['CON HERIDOS', 'CON VICTIMAS']

# Concatena las columnas seleccionadas
nueva_columna = pd.concat([df_final[col] for col in columnas_a_combinar], ignore_index=True)

# Si quieres eliminar valores nulos antes de concatenar, puedes hacerlo así:
nueva_columna = pd.concat([df_final[col].dropna() for col in columnas_a_combinar], ignore_index=True)

# Elimina las columnas originales si ya no las necesitas
df_final = df_final.drop(columns=columnas_a_combinar)

# Luego, añade la nueva columna al GeoDataFrame
df_final['CON HERIDOS'] = nueva_columna


In [ ]:
# Selecciona las columnas que quieres combinar
columnas_a_combinar = ['SIN HERIDOS', 'SIN VICTIMAS']

# Concatena las columnas seleccionadas
nueva_columna = pd.concat([df_final[col] for col in columnas_a_combinar], ignore_index=True)

# Si quieres eliminar valores nulos antes de concatenar, puedes hacerlo así:
nueva_columna = pd.concat([df_final[col].dropna() for col in columnas_a_combinar], ignore_index=True)

# Elimina las columnas originales si ya no las necesitas
df_final = df_final.drop(columns=columnas_a_combinar)

# Luego, añade la nueva columna al GeoDataFrame
df_final['SIN HERIDOS'] = nueva_columna


In [ ]:
# Selecciona las columnas que quieres combinar
columnas_a_combinar = ['ADULTOS', 'MAYORES DE EDAD']

# Concatena las columnas seleccionadas
nueva_columna = pd.concat([df_final[col] for col in columnas_a_combinar], ignore_index=True)

# Si quieres eliminar valores nulos antes de concatenar, puedes hacerlo así:
nueva_columna = pd.concat([df_final[col].dropna() for col in columnas_a_combinar], ignore_index=True)

# Elimina las columnas originales si ya no las necesitas
df_final = df_final.drop(columns=columnas_a_combinar)

# Luego, añade la nueva columna al GeoDataFrame
df_final['ADULTOS'] = nueva_columna


In [ ]:
# Selecciona las columnas que quieres combinar
columnas_a_combinar = ['MENORES', 'MENORES DE 18 AÑOS']

# Concatena las columnas seleccionadas
nueva_columna = pd.concat([df_final[col] for col in columnas_a_combinar], ignore_index=True)

# Si quieres eliminar valores nulos antes de concatenar, puedes hacerlo así:
nueva_columna = pd.concat([df_final[col].dropna() for col in columnas_a_combinar], ignore_index=True)

# Elimina las columnas originales si ya no las necesitas
df_final = df_final.drop(columns=columnas_a_combinar)

# Luego, añade la nueva columna al GeoDataFrame
df_final['MENORES'] = nueva_columna


A continuación asignamos una variable para el shapefile que contiene las geometrías de los distritos. Después exploramos este shapefile para ver a partir de qué columnas podemos unir los datos.

In [ ]:
distritos = gpd.read_file(distritos_shp)

In [ ]:
print(distritos.head())

In [ ]:
print(distritos.columns)

Al comparar los datos del shapefile y la base de datos vemos que hay columnas que comparten información pero no se llaman igual, por lo que cambiamos el nombre de una del shapefile para realizar la unión.

In [ ]:
# Renombrar la columna 'shapefile_ID' a 'df_ID' para que coincidan
distritos = distritos.rename(columns={'DISTRI_MT': 'DISTRITOS'})

Nos aseguramos que ambas columnas están en el mismo formato para realizar la unión.

In [ ]:
df_final['DISTRITOS'] = df_final['DISTRITOS'].astype(str)
distritos['DISTRITOS'] = distritos['DISTRITOS'].astype(str)

Realizamos la unión a partir de la columna que comparte información.

In [ ]:
# Realizar la unión entre el DataFrame y el shapefile
df_union = df_final.merge(distritos[['DISTRITOS', 'geometry']], on='DISTRITOS', how='left')

Una vez realizada la unión, comenzamos a modificar la base de datos para poder trabajar con ella. En primer lugar, generamos una columna de año y otra de mes a partir de la columna generada a partir del nombre de los archivos.

In [ ]:
# Con este código separamos la columna fecha en dos nuevas columnas, año y mes.
df_union[['Mes', 'Año']] = df_union['Fecha'].str.split('_', expand=True)

# Mostrar las primeras filas del GeoDataFrame con las nuevas columnas
print(df_union[['Fecha', 'Mes', 'Año']])

Después generamos una columna con los meses como variable numérica. Tener la variable de meses de este modo puede llegar a ser útil de cara a algunas representaciones gráficas de los datos.

In [ ]:
# Diccionario que mapea los meses en palabras a su respectivo número
mes_a_numero = {
    'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6,
    'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
}

# Convertir la columna 'mes' de nombre a número
df_union['mes'] = df_union['Mes'].map(mes_a_numero)

A continuación, convertimos el data frame en un geodataframe.

In [ ]:
# Asegurarse de que es un GeoDataFrame
df_union = gpd.GeoDataFrame(df_union, geometry='geometry')

Después añadimos algunas columnas que podrían ser útiles a la hora de estudiar la base de datos, como el área de cada distrito.

In [ ]:
df_union['Area'] = df_union.geometry.area # Por defecto se calcula el área en metros cuadrados
df_union['Area_ha'] = df_union['Area'] / 10000 # Área en hectáreas
df_union['Area_km2'] = df_union['Area'] / 1000000 # Área en kilómetros cuadrados


Luego, eliminamos las columnas que no nos interesan de nuestra base de datos.

In [ ]:
#He puesto esto como ejemplo, pero se pueden poner otros o ninguno, depende de lo que acabemos haciendo.
df_union = df_union.drop(columns=['Fecha','DENUNCIAS','DETENIDOS E IMPUTADOS','DETENIDOS E INVESTIGADOS'])

Después de esto, incorporamos datos de población a nuestro dataframe. Como se puede ver a continuación, dan información sobre la población por distrito, lo cual podría ser una variable interesante a la hora de estudiar los delitos cometidos.

In [ ]:
# Leer el archivo CSV y seleccionar las primeras 22 columnas, que son las que contienen los datos que nos interesan
df_poblacion = pd.read_csv(poblacion, sep=";").head(21)

# Mostrar las primeras filas del DataFrame para verificar
df_poblacion.head()
df_poblacion.info()

Después cambiamos el nombre de algunas columnas para que sea más facil trabajar con ellas en el futuro. Además, eliminamos las columnas que no nos interesan.

In [ ]:
# Cambiar nombres de algunas columnas para adecuarlos
df_poblacion = df_poblacion.rename(columns={'num_personas': 'poblacion'})
df_poblacion = df_poblacion.rename(columns={'num_personas_hombres': 'pob_hombres'})
df_poblacion = df_poblacion.rename(columns={'num_personas_mujeres': 'pob_mujeres'})
df_poblacion = df_poblacion.rename(columns={'distrito': 'DISTRITOS'})

# Eliminar columnas que no resultan interesantes en el analisis
df_poblacion = df_poblacion.drop(columns=['fecha', 'cod_municipio', 'municipio',
                                          'cod_barrio', 'barrio'])

Antes de poder unir estos datos con el resto necesitamos modificar el nombre de la columna de distritos para que coincidan.

In [ ]:
# Poner los nombres de los distritos en mayúsculas, también para manejar la unión
df_poblacion['DISTRITOS'] = df_poblacion['DISTRITOS'].str.upper()
df_poblacion['DISTRITOS'] = df_poblacion['DISTRITOS'].astype(str)

In [ ]:
# Eliminar espacios después de los guiones en los nombres de los distritos
# para mantener el mismo nombre y poder realizar la unión correctamente
df_poblacion['DISTRITOS'] = df_poblacion['DISTRITOS'].str.replace(r'\s*-\s*', '-', regex=True)
df_union['DISTRITOS'] = df_union['DISTRITOS'].str.replace(r'\s*-\s*', '-', regex=True)

df_poblacion['DISTRITOS'] = df_poblacion['DISTRITOS'].str.strip().str.upper().str.replace(r'\s*-\s*', '-', regex=True)
df_union['DISTRITOS'] = df_union['DISTRITOS'].str.strip().str.upper().str.replace(r'\s*-\s*', '-', regex=True)

Realizamos la unión y comprobamos que se ha realizado correctamente.

In [ ]:
df_union = df_union.merge(df_poblacion, on='DISTRITOS', how='left')

In [ ]:
df_union.head()

Después convertimos los datos de población a tipo numérico y calculamos la densidad de población (total, de hombres y de mujeres).

In [ ]:
# Convertir la columna 'Poblacion' a tipo numérico (por si tiene valores tipo str)
df_union['poblacion'] = pd.to_numeric(df_union['poblacion'], errors='coerce')
df_union['pob_hombres'] = pd.to_numeric(df_union['pob_hombres'], errors='coerce')
df_union['pob_mujeres'] = pd.to_numeric(df_union['pob_mujeres'], errors='coerce')

# Calcular densidad de población
df_union['densidad_pob_km2'] = df_union['poblacion'] / df_union['Area_km2']
df_union['densidad_pob_ha'] = df_union['poblacion'] / df_union['Area_ha']
df_union['densidad_pob_hombres'] = df_union['pob_hombres'] / df_union['Area_km2']
df_union['densidad_pob_mujeres'] = df_union['pob_mujeres'] / df_union['Area_km2']
df_union['densidad_pob_hombres_ha'] = df_union['pob_hombres'] / df_union['Area_ha']
df_union['densidad_pob_mujeres_ha'] = df_union['pob_mujeres'] / df_union['Area_ha']

Después pasamos a la limpieza de los datos. Para ello primero comprobamos si hay datos duplicados o valores nulos en nuestro dataframe.

In [ ]:
#Este código muestra las filas duplicadas.
df_union[df_union.duplicated(keep='first')]

In [ ]:
#Esta función nos permite saber si hay datos nulos
df_union.isnull().sum()

Como en las bases de datos originales había información sobre el total de la zona y sabiendo que estos datos no poseen geometrías y tampoco datos de población, aparecen como nulos. Por ello los eliminaremos.

In [ ]:
df_union = df_union.dropna()

In [ ]:
# Desactivar los warnings
warnings.simplefilter('ignore')

# Nos aseguramos de que las columnas de 'Año' y 'mes' son de tipo string.
df_union['Año'] = df_union['Año'].astype(str)
df_union['mes'] = df_union['mes'].astype(str)

# Crear una nueva columna combinada 'Año-Mes' para la animación.
df_union['Año-mes'] = df_union['Año'] + '-' + df_union['mes']

# Convertir la columna 'Año-Mes' a tipo fecha para ordenar cronológicamente.
df_union['Fecha'] = pd.to_datetime(df_union['Año-mes'], format='%Y-%m')

In [ ]:
df_union.head()

Hasta aquí no hay ningún cambio respecto al EDA realizado con estos mismos datos, pero como nos interesa realizar también un análisis de patrones de puntos vamos a cargar otros datos. En este caso de multas en Madrid. Una vez cargados estos datos vamos a limpiarlos y modificarlos para poder trabajar con ellos

Primero establecemos la ruta del archivo y creamos un dataframe con los datos de multas.

In [ ]:
# Ruta del archivo CSV con los datos de multas de tráfico en mayo de 2024
multas = os.path.join(folder, 'Multas_5_2024', '202405detalle.csv')

# Cargar el CSV con la codificación correcta
df_multas = pd.read_csv(multas, sep=';', encoding='ISO-8859-1', dtype={'COORDENADA-X': str, 'COORDENADA-Y': str})

Limpiamos los nombres de las columnas eliminando caracteres no deseados y también eliminamos valores nulos. Además convertimos las columnas de coordenadas a valores numéricos para poder trabajar con ellas.

In [ ]:
# Limpiar nombres de columnas
df_multas.columns = df_multas.columns.str.strip()

# Eliminar valores nulos o vacíos en 'COORDENADA-X' y 'COORDENADA-Y'
df_multas = df_multas.dropna(subset=['COORDENADA-X', 'COORDENADA-Y'])
df_multas = df_multas[(df_multas['COORDENADA-X'] != '') & (df_multas['COORDENADA-Y'] != '')]

# Convertir coordenadas a tipo numérico
df_multas['COORDENADA-X'] = pd.to_numeric(df_multas['COORDENADA-X'], errors='coerce')
df_multas['COORDENADA-Y'] = pd.to_numeric(df_multas['COORDENADA-Y'], errors='coerce')

# Eliminar posibles valores NaN generados
df_multas = df_multas.dropna(subset=['COORDENADA-X', 'COORDENADA-Y'])

A continuación, creamos las geometrías de los puntos según las coordenadas del dataframe.

In [ ]:
# Crear geometría de puntos con CRS 25830
geometry = gpd.points_from_xy(df_multas['COORDENADA-X'], df_multas['COORDENADA-Y'])
gdf_multas = gpd.GeoDataFrame(df_multas, geometry=geometry, crs="EPSG:25830")

Recragamos la capa de distritos para que no cuente con las modificaciones que hemos realizado previamente, nos aseguramos de que el sistema de referencia de multas y distritos coincida y unimos los datos.

In [ ]:
# Cargar shapefile de distritos
distritos = gpd.read_file(distritos_shp)

# Establecer mismo SRC para las dos capas (EPSG:4326)
gdf_multas = gdf_multas.to_crs(epsg=4326)
distritos = distritos.to_crs(epsg=4326)

# Unión espacial para seleccionar solo las multas dentro de los distritos
gdf_multas = gpd.sjoin(gdf_multas, distritos, how="inner", predicate="within")

Eliminamos algunas columnas que no nos interesan para el estudio y cambiamos el nombre de otras para facilitarnos el trabajo más adelante. Por último, visualizamos los datos.

In [ ]:
# Eliminar las columnas innecesarias
columnas_a_eliminar = ['NOMBRE', 'DISTRI_MT', 'FCH_ALTA', 'FCH_BAJA', 'OBSERVACIO', 'ACUERDO', 'COD_DIS', 'COD_DIS_TX', 'index_right', 'Shape_Leng', 'Shape_Area']
gdf_multas = gdf_multas.drop(columns=columnas_a_eliminar, errors='ignore')

# Cambiar el nombre de la columnas de distritos y coordenadas
gdf_multas = gdf_multas.rename(columns={'DISTRI_MAY': 'DISTRITO'})
gdf_multas = gdf_multas.rename(columns={'COORDENADA-X': 'LONGITUD'})
gdf_multas = gdf_multas.rename(columns={'COORDENADA-Y': 'LATITUD'})

# Cambiar el nombre de la columna 'DISTRI_MAY' a 'DISTRITO' en distritos para que sea igual en ambos casos
distritos = distritos.rename(columns={'DISTRI_MAY': 'DISTRITO'})

gdf_multas.info()
gdf_multas.head()
distritos.info()
distritos.head()

# 4. RESULTADOS

En este notebook realizamos un ESDA a partir de unos datos que fueron explorados previamente en un EDA. En ese EDA se observó que el número de delitos no guardaba una relación estadísticamente significativa con variables como el área o la población. Por eso las variables del número de delitos se estudian en este ESDA sin relacionarlar con el área y la población. Si quiere acceder al EDA previo puede verlo [aquí.](https://github.com/RoberRodero/Delitos-Madrid)

Dentro de este ESDA los apartados de autocorrelación global y local, clusterización, regionalización y uso de algoritmos de clasificación se realizarán a partir de los datos explorados en el EDA anterior mientras que el análisis de patrones de puntos se realizará a partir de los datos de multas.

Para comenzar el análisis de autocorrelación espacial calculamos la matriz de pesos. Como los distritos aparecen repetidos para cada fecha y su distribución espacial no cambia con el tiempo solo se calculan estos pesos una vez. En el resultado que mostramos podemos ver cuantos distritos tiene como vecino inmediato cada uno de los distritos de la ciudad de Madrid.

In [ ]:
# Eliminar duplicados basados en la columna de 'DISTRITOS'
df_union_unique = df_union.drop_duplicates(subset='DISTRITOS')

# Calcular la matriz de pesos con los distritos únicos
w = weights.Queen.from_dataframe(df_union_unique, ids="DISTRITOS")

# Ver los pesos
print(w.weights)

Por ejemplo, aquí podemos ver los distritos que son vecinos del distrito de Latina.

In [ ]:
w['LATINA']

Una vez calculada la matriz de pesos, normalizamos los valores ya que esto mejora los resultados de los cálculos que realizaremos posteriormente.

In [ ]:
# Estandarizar la matriz de pesos
w.transform = 'R'

A continuacuón se puede ver como han cambiado los valores de los pesos.

In [ ]:
w['LATINA']

Después, mostramos el nombre de los distristos vecinos de cada distrito.

In [ ]:
# Verifica los vecinos de cada distrito en la matriz de pesos
print(w.neighbors)

Ahora que tenemos la matriz de pesos, podemos representar de manera gráfica la conectividad entre los distritos.

In [ ]:
plot_spatial_weights(w, df_union_unique, indexed_on="DISTRITOS")

Como en nuestra base de datos contamos con información sobre varios tipos de delitos en diferentes fechas, calculamos los pesos para cada tipo de delito y fecha para poder analizar la autocorrelación espacial de cualquiera de ellos en cualquier momento. También calculamos los valores de la desviación estándar de los delitos y de la desviación estándar de los pesos de los delitos. Estas desviaciones estándar se calculan para que la magnitud de los campos no afecte al cálculo de la autocorrelación espacial. A continuación mostramos los resultados de los pesos según la fecha y el delito para cada distrito para comprobar que el resultado obtenido es coherente.

In [ ]:
# Lista de delitos a procesar
delitos = [
    'RELACIONADAS CON LAS PERSONAS',
    'RELACIONADAS CON EL PATRIMONIO',
    'POR TENENCIA DE ARMAS',
    'POR TENENCIA DE DROGAS',
    'POR CONSUMO DE DROGAS',
    'PROPIEDAD INTELECTUAL E INDUSTRIAL',
    'INFRACCIONES ALIMENTARIAS',
    'INSPECCIONES Y ACTUACIONES',
    'CON HERIDOS',
    'SIN HERIDOS',
    'ADULTOS',
    'MENORES'
]

# Iterar sobre cada fecha única
for fecha in df_union['Fecha'].unique():
    # Filtrar los datos para esa fecha
    df_fecha = df_union[df_union['Fecha'] == fecha].copy()  # Usamos .copy() para evitar el SettingWithCopyWarning

    # Iterar sobre cada campo en la lista de delitos
    for delito in delitos:
        # 1. Calcular la desviación estándar normalizada para el campo
        df_fecha[f'{delito}_std'] = (df_fecha[delito] - df_fecha[delito].mean()) / df_fecha[delito].std()

        # 2. Aplicar el desfase espacial (lag espacial) sobre el campo original
        df_fecha[f'w_{delito}'] = weights.lag_spatial(w, df_fecha[delito].values)

        # 3. Aplicar el desfase espacial (lag espacial) sobre la desviación estándar
        df_fecha[f'w_{delito}_std'] = weights.lag_spatial(w, df_fecha[f'{delito}_std'].values)

        # 4. Usar .loc para actualizar el DataFrame original con las columnas calculadas
        df_union.loc[df_union['Fecha'] == fecha, f'{delito}_std'] = df_fecha[f'{delito}_std']
        df_union.loc[df_union['Fecha'] == fecha, f'w_{delito}'] = df_fecha[f'w_{delito}']
        df_union.loc[df_union['Fecha'] == fecha, f'w_{delito}_std'] = df_fecha[f'w_{delito}_std']

# Ver los resultados
print(df_union[['Fecha', 'DISTRITOS'] + [f'w_{delito}' for delito in delitos] + [f'w_{delito}_std' for delito in delitos]])

Después generamos una gráfica en la que mostramos la desviación estándar de cada delito frente a la desviación estandar de los pesos de cada delito. Esta gráfica se divide en cuatro cuadrantes y en función de en que cuadrante se situe cada dato se indica la relación que tiene con los datos vecinos, si se situa en el cuadrante de arriba a la izquierda quiere decir que es un valor bajo rodeado de valores altos, si se situa en el cuadrante de arriba a la derecha es un valor alto rodeado de valores altos, si se situa en el cuadrante de abajo a la derecha es un valor alto rodeado de valores bajos y si se situa en el cuadrante de abajo a la izquierda es un valor bajo rodeado de valores bajos. Además también aparece un línea que indica si la autocorrelación espacial es positiva (linea creciente hacia la derecha), negativa (linea decreciente hacia la derecha) o inexistente (linea horizontal).

Para ilustrar un ejemplo, vamos a estudiar los resultados obtenidos con el tipo de delito Relacionados con las personas para enero de 2015 (primera fecha que aparece en el selector). Se puede observar que en el caso de que exista autocorrelación espacial estadísticamente significativa en estos datos, será de tipo negativo.

In [ ]:
# Formatear las fechas para mostrar solo día, mes y año
df_union['Fecha_formateada'] = df_union['Fecha'].dt.strftime('%d-%m-%Y')

# Obtener las fechas únicas, ordenarlas y quitar duplicados
fechas_unicas = df_union['Fecha_formateada'].unique()
fechas_unicas = sorted(fechas_unicas)

# Crear un desplegable para seleccionar la fecha
fecha_selector = widgets.Dropdown(
    options=fechas_unicas,
    description='Fecha:',
    disabled=False
)

# Crear un desplegable para seleccionar el delito
delito_selector = widgets.Dropdown(
    options=delitos,
    description='Delito:',
    disabled=False
)

# Función para actualizar el gráfico y calcular el índice de Moran
def actualizar_grafico(fecha, delito):
    # Filtrar los datos solo para la fecha seleccionada
    df_fecha = df_union[df_union['Fecha_formateada'] == fecha].copy()  # Usamos .copy() para evitar el SettingWithCopyWarning

    # Calcular la desviación estándar normalizada para el delito seleccionado solo para esa fecha
    df_fecha[f'{delito}_std'] = (df_fecha[delito] - df_fecha[delito].mean()) / df_fecha[delito].std()

    # Crear la figura y el eje para el gráfico
    f, ax = plt.subplots(1, figsize=(9, 9))

    # Graficar los datos
    sns.regplot(x=f'{delito}_std', y=f'w_{delito}_std', data=df_fecha, ci=None, ax=ax)

    # Añadir líneas verticales y horizontales
    ax.axvline(0, c='k', alpha=0.5)
    ax.axhline(0, c='k', alpha=0.5)

    # Etiquetas de las regiones (HH, HL, LH, LL)
    texts = []
    texts.append(ax.text(2, 0.8, "HH", fontsize=25))
    texts.append(ax.text(2, -0.5, "HL", fontsize=25))
    texts.append(ax.text(-1, 0.5, "LH", fontsize=25))
    texts.append(ax.text(-1, -0.5, "LL", fontsize=25))

    # Ajustar las posiciones de las etiquetas para evitar que se solapen con puntos
    adjust_text(texts, ax=ax)

    # Mostrar el gráfico
    plt.title(f'Gráfico de {delito} con fecha {delito}')
    plt.show()

# Mostrar la interacción entre widgets y gráfico
widgets.interactive(actualizar_grafico, fecha=fecha_selector, delito=delito_selector)


Aparte de un gráfico como el anterior, también generamos el valor del índice de Moran. Este índice indica si existe autocorrelación espacial global en los datos que estemos estudiando. Valores cercanos a 1 indican que la autocorrelación espacial es positiva, valores cercanos a -1 indican que la autocorrelación espacial es negativa y valores cercanos a 0 indican que no existe autocorrelación espacial. También se muestra el p-valor, que nos dice si el patrón espacial observado es producto del azar (p-valor > 0,05) o si realmente existe una autocorrelación espacial significativa (p-valor < 0,05).

Continuando con el ejemplo que estamos explicando, podemos observar que la autocorrelación espacial negativa no es estadísticamente significativa.





In [ ]:
# Crear un desplegable para seleccionar la fecha
fecha_selector = widgets.Dropdown(
    options=fechas_unicas,
    description='Fecha:',
    disabled=False
)

# Crear un desplegable para seleccionar el delito
delito_selector = widgets.Dropdown(
    options=delitos,
    description='Delito:',
    disabled=False
)

# Función para actualizar el gráfico y calcular el índice de Moran
def actualizar_grafico(fecha, delito):
    # Filtrar los datos para la fecha seleccionada
    df_fecha = df_union[df_union['Fecha_formateada'] == fecha].copy()  # Usamos .copy() para evitar el SettingWithCopyWarning

    # Calcular el índice de Moran para el campo seleccionado
    moran = esda.Moran(df_fecha[delito], w)

    # Mostrar el índice de Moran
    print(f'Índice de Moran para el campo "{delito}" en la fecha {fecha}:')
    print(f'Moran I: {moran.I}')
    print(f'P-valor: {moran.p_sim}')

    plot_moran(moran)

# Mostrar los widgets y actualizar el gráfico en función de la selección
widgets.interactive(actualizar_grafico, fecha=fecha_selector, delito=delito_selector)

Hasta ahora solo hemos estudiado la autocorrelación espacial global, pero también se puede estudiar la autocorrelación espacial local. Para analizar si existe autocorrelación local o no y de que tipo es hay que calcular los valores LISA. Lo calculamos para todos los delitos y todas la fechas asignando cada distrito a un cuadrante (los de las gráficas de autocorrelación espacial global) y si es estadísticamente significativo.

In [ ]:
# Iterar sobre cada fecha única
for fecha in df_union['Fecha'].unique():
    # Filtrar los datos para esa fecha
    df_fecha = df_union[df_union['Fecha'] == fecha].copy()

    # Iterar sobre cada delito en la lista de delitos
    for delito in delitos:
        # Calcular el Moran Local para el delito seleccionado
        lisa = esda.Moran_Local(df_fecha[delito], w)

        # Añadir una nueva columna 'significant' que será True si p-valor es menor a 0.05
        df_fecha['significant'] = lisa.p_sim < 0.05

        # Añadir una nueva columna 'quadrant' con los cuadrantes
        df_fecha['quadrant'] = lisa.q

        # Asignar los valores de 'significant' y 'quadrant' al DataFrame original (df_union) usando .loc
        df_union.loc[df_union['Fecha'] == fecha, f'{delito}_significant'] = df_fecha['significant']
        df_union.loc[df_union['Fecha'] == fecha, f'{delito}_quadrant'] = df_fecha['quadrant']

# Verificar los resultados
print(df_union[['Fecha', 'DISTRITOS'] + [f'{delito}_significant' for delito in delitos] + [f'{delito}_quadrant' for delito in delitos]])


Una vez contamos con los datos LISA podemos representarlo gráficamente. En este tipo de gráficos podemos ver los distritos representados según el cuadrante en el que han sido incluidos. Hay varias opciones:

Que los distritos con valores altos se encuentren cerca entre sí, que los distritos con valores bajos se encuentren cerca entre sí, que distritos con valores altos se encuentran cerca de distritos con valores bajos o que distritos con valores bajos se encuentren cerca de distritos con valores altos.

 Para el ejemplo que teníamos en la autocorrelación global, podemos ver como en algunos ditritos existe autocorrelación local, de los tipos alto-alto y bajo-alto. Este resultado podría dar lugar a un estudio más profundo acerca de la causa de esta autocorrelación local. Además, al tener datos para muchas fechas podríamos comparar entre distintos momentos en el tiempo, lo que ayudaría a encontrar las causas de este fenómeno.

In [ ]:
# Desactivar los warnings
warnings.simplefilter('ignore')

# Crear un desplegable para seleccionar la fecha
fecha_selector = widgets.Dropdown(
    options=fechas_unicas,
    description='Fecha:',
    disabled=False
)

# Crear un desplegable para seleccionar el campo
delito_selector = widgets.Dropdown(
    options=delitos,
    description='Delito:',
    disabled=False
)

# Función para actualizar el gráfico y calcular el índice de Moran
def actualizar_grafico(fecha, delito):
    # Filtrar los datos solo para la fecha seleccionada
    df_fecha = df_union[df_union['Fecha_formateada'] == fecha].copy()

    # Crear el gráfico
    f, ax = plt.subplots(1, figsize=(9, 9))

    # Mostrar los grupos no significativos
    ns = df_fecha.loc[df_fecha[f'{delito}_significant'] == False, 'geometry']
    ns.plot(ax=ax, color='k', label='No significativo')

    # Mostrar los grupos HH
    hh = df_fecha.loc[(df_fecha[f'{delito}_quadrant'] == 1) & (df_fecha[f'{delito}_significant'] == True), 'geometry']
    hh.plot(ax=ax, color='red', label='Alto-Alto')

    # Mostrar los grupos LL
    ll = df_fecha.loc[(df_fecha[f'{delito}_quadrant'] == 3) & (df_fecha[f'{delito}_significant'] == True), 'geometry']
    ll.plot(ax=ax, color='blue', label='Bajo-Bajo')

    # Mostrar los grupos LH
    lh = df_fecha.loc[(df_fecha[f'{delito}_quadrant'] == 2) & (df_fecha[f'{delito}_significant'] == True), 'geometry']
    lh.plot(ax=ax, color='#83cef4', label='Bajo-Alto')

    # Mostrar los grupos HL
    hl = df_fecha.loc[(df_fecha[f'{delito}_quadrant'] == 4) & (df_fecha[f'{delito}_significant'] == True), 'geometry']
    hl.plot(ax=ax, color='#e59696', label='Alto-Bajo')

    # Agregar leyenda
    legend_patches = [
        mpatches.Patch(color='red', label='Alto-Alto'),
        mpatches.Patch(color='blue', label='Bajo-Bajo'),
        mpatches.Patch(color='#83cef4', label='Bajo-Alto'),
        mpatches.Patch(color='#e59696', label='Alto-Bajo'),
        mpatches.Patch(color='k', label='No significativo')
    ]
    ax.legend(handles=legend_patches, loc='lower right')

    # Estilo y visualización
    f.suptitle(f'LISA de {delito} en {fecha}', size=16)
    f.set_facecolor('0.75')
    ax.set_axis_off()
    plt.show()

# Mostrar la interacción entre widgets y gráfico
widgets.interactive(actualizar_grafico, fecha=fecha_selector, delito=delito_selector)


También podemos representarlo junto al diagrama de dispersión de moran y los valores del delito por distrito en una fecha determinada para tener una visión más completa de los datos.

In [ ]:
# Crear un desplegable para seleccionar la fecha
fecha_selector = widgets.Dropdown(
    options=fechas_unicas,
    description='Fecha:',
    disabled=False
)

# Crear un desplegable para seleccionar el delito
campo_selector = widgets.Dropdown(
    options=delitos,
    description='delito:',
    disabled=False
)

# Función para actualizar el gráfico y calcular el índice de Moran
def actualizar_grafico(fecha, delito):
    # Filtrar los datos solo para la fecha seleccionada
    df_fecha = df_union[df_union['Fecha_formateada'] == fecha].copy()  # Usamos .copy() para evitar el SettingWithCopyWarning

    lisa2 = esda.Moran_Local(df_fecha[delito], w)

    plot_local_autocorrelation(lisa2, df_fecha, df_fecha[delito])

# Mostrar la interacción entre widgets y gráfico
widgets.interactive(actualizar_grafico, fecha=fecha_selector, delito=delito_selector)

# Clustering y regionalización.


A continuación, para seguir con el ESDA, en este apartado vamos a agrupar los distritos en clusters y también a regionalizar y comparar la diferencia en las agrupaciones generadas en función de los delitos y las divisiones administrativas reales. Como tenemos datos para cada mes durante una década, permitimos al usuario que elija para que periodo de tiempo desea ver estas agrupaciones.

Para ello en primer lugar vamos a hacer un ejemplo para 5 clusters. Mas adelante veremos un par de métodos para seleccionar el número óptimo de clusters para los datos de estudio pero realizar un primer ejemplo nos parece interesante para una primera visualización de los resultados.

Como resultado del siguiente código podemos ver a que cluster se ha asignado cada distrito para cada fecha.

In [ ]:
# Definir el modelo KMeans, determinando el número de clusters y una seed
kmeans5 = KMeans(n_clusters=5, random_state=12345, n_init=10)

# Iterar sobre cada fecha única
for fecha in df_union['Fecha'].unique():
    # Filtrar los datos para esa fecha
    df_fecha = df_union[df_union['Fecha'] == fecha].copy()

    # Ejecutar el algoritmo de clustering
    k5clust = kmeans5.fit(df_fecha[delitos])

    # Guardar los clusters en la columna 'k5clust' en df_union
    df_union.loc[df_union['Fecha'] == fecha, 'k5clust'] = k5clust.labels_

# Ver los primeros resultados
print(df_union[['Fecha', 'k5clust']].head())



Una vez asignamos cada distrito a una grupo o cluster podemos representar gráficamente el resutado. Podemos ver que como los datos de delitos son diferentes para cada fecha, las agrupaciones van cambiando dependiendo del momento en el que estudiemos los grupos.

Tomando como ejemplo la misma fecha que al estudiar la autocorrelación local y global, podemos ver como existe un gran grupo en el norte y Este de la ciudad y el resto de grupos se dividen entre los distritos del centro, sur y oeste.

In [ ]:
# Obtener fechas únicas, ordenarlas y formatearlas como "DD/MM/AAAA"
fechas_unicas = sorted(df_union['Fecha'].dt.strftime('%Y/%m/%d').unique())

# Crear un widget de selección de fecha
fecha_selector = widgets.Dropdown(
    options=fechas_unicas,
    description="Fecha:",
    style={'description_width': 'initial'}
)

# Función para actualizar el mapa según la fecha seleccionada
def actualizar_mapa(fecha_str):
    # Convertir la fecha seleccionada de string a datetime
    fecha = pd.to_datetime(fecha_str, format='%Y/%m/%d')


    # Filtrar el DataFrame por la fecha seleccionada
    df_filtrado = df_union[df_union['Fecha'] == fecha]

    # Limpiar la figura anterior
    plt.clf()

    # Crear nueva figura
    f, ax = plt.subplots(1, figsize=(18, 9))

    # Graficar mapa
    df_filtrado.plot(
        column='k5clust', categorical=True, legend=True, linewidth=0, ax=ax
    )

    # Quitar ejes y añadir título
    ax.set_axis_off()
    plt.title(f'Clasificación geodemográfica por delitos - {fecha_str}')

    # Mostrar el mapa
    plt.show()

# Usar interact para actualizar dinámicamente
widgets.interactive(actualizar_mapa, fecha_str=fecha_selector)


A continuación mostramos el mismo resultado en forma de tabla, permitiendo al usuario seleccionar que fecha quiere ver.

In [ ]:
# Crear un widget de selección de fecha
fecha_selector = widgets.Dropdown(
    options=fechas_unicas,
    description="Fecha:",
    style={'description_width': 'initial'}
)

# Crear un widget de salida para mostrar los resultados
output = widgets.Output()

# Función para calcular el tamaño de los clusters de KMeans por fecha seleccionada
def calcular_tamano_clusters(fecha_str):
    # Limpiar la salida anterior
    with output:
        clear_output(wait=True)
    # Convertir la fecha seleccionada de string a datetime
    fecha = pd.to_datetime(fecha_str, format='%Y/%m/%d')

    # Filtrar el DataFrame por la fecha seleccionada
    df_filtrado = df_union[df_union['Fecha'] == fecha]

    # Calcular el tamaño de los clusters
    kmeans5sizes = df_filtrado.groupby('k5clust').size()

    # Mostrar el resultado
    with output:
        print(f"Tamaño de los clusters para la fecha: {fecha_str}")
        display(kmeans5sizes)

# Usar interactive_output para evitar la firma de la función
widgets.interactive_output(calcular_tamano_clusters, {'fecha_str': fecha_selector})
display(fecha_selector, output)

También mostramos estos resultados como gráfica de barras, permitiendo una vez mas al usuario seleccionar que fecha quiere ver.

In [ ]:
# Crear un widget de selección de fecha
fecha_selector = widgets.Dropdown(
    options=fechas_unicas,
    description="Fecha:",
    style={'description_width': 'initial'}
)
# Función para actualizar el mapa según la fecha seleccionada
def barras(fecha_str):
  # Convertir la fecha seleccionada de string a datetime
    fecha = pd.to_datetime(fecha_str, format='%Y/%m/%d')


    # Filtrar el DataFrame por la fecha seleccionada
    df_filtrado = df_union[df_union['Fecha'] == fecha]

    # Calcular el tamaño de los clusters
    kmeans5sizes = df_filtrado.groupby('k5clust').size()

    bar_kmeans5 = kmeans5sizes.plot.bar()

    # Mostrar el gráfico actualizado
    plt.show()

# Usar interact para actualizar dinámicamente
widgets.interactive(barras, fecha_str=fecha_selector)

Para terminar con el ejemplo de 5 clusters, mostramos el número de delitos medios cometidos en cada clúster. Ya a partir de este resultado podemos ver como probablemente tomar 5 clústers no es óptimo ya que en algunos de ellos el número medio de delitos es muy similar. Si lo comprobamos para varias fechas podemos ver como este problema se repite.

In [ ]:
# Crear un widget de selección de fecha
fecha_selector = widgets.Dropdown(
    options=fechas_unicas,
    description="Fecha:",
    style={'description_width': 'initial'}
)

# Crear un widget de salida para mostrar los resultados
output = widgets.Output()

# Función para calcular el tamaño de los clusters de KMeans por fecha seleccionada
def tabla(fecha_str):
    # Limpiar la salida anterior
    with output:
        clear_output(wait=True)
    # Convertir la fecha seleccionada de string a datetime
    fecha = pd.to_datetime(fecha_str, format='%Y/%m/%d')

    # Filtrar el DataFrame por la fecha seleccionada
    df_filtrado = df_union[df_union['Fecha'] == fecha]

    # Calcular el número de delitos por tipo de los clusters
    k5meanss = df_filtrado.groupby('k5clust')[delitos].mean()

    # Mostrar el resultado
    with output:
        print(f"Tamaño de los clusters para la fecha: {fecha_str}")
        display(k5meanss.T)

# Usar interactive_output para evitar la firma de la función
widgets.interactive_output(tabla, {'fecha_str': fecha_selector})
display(fecha_selector, output)

Para continuar con la clusterización vamos a utilizar un método diferente, AgglomerativeClustering, que aparte de permitirnos agrupar en el número de cluster que deseemos (al igual que hacía Kmeans), también nos permite generar un dendograma en el que ver si el número de clúster seleccionado es idóneo o no. Para comenzar a utilizar este ejemplo hemos decidido hacerlo en esta ocasion con 8 clusters como punto de partido, generando el mapa que se ve a continuación.

Si comparamos con el ejemplo utilizando 5 clusters podemos ver las diferencias que genera utilizar un número diferente de clusters para hacer la agrupación.

In [ ]:
# Crear un widget de selección de fecha
fecha_selector = widgets.Dropdown(
    options=fechas_unicas,
    description="Fecha:",
    style={'description_width': 'initial'}
)
# Función para actualizar el mapa según la fecha seleccionada
def agregar(fecha_str):
  # Convertir la fecha seleccionada de string a datetime
    fecha = pd.to_datetime(fecha_str, format='%Y/%m/%d')


    # Filtrar el DataFrame por la fecha seleccionada
    df_filtrado = df_union[df_union['Fecha'] == fecha]

    s_agg13 = AgglomerativeClustering(n_clusters=8, connectivity=w.sparse)

    # Ejecutar el algoritmo de clustering
    s_agg13clust = s_agg13.fit(df_filtrado[delitos])
    df_filtrado['sagg13clust'] = s_agg13clust.labels_

    # Crear gráfico
    f, ax = plt.subplots(1, figsize=(9, 9))

    # Mostrar los valores
    df_filtrado.plot(
        column='sagg13clust', categorical=True, legend=True, linewidth=0, ax=ax
    )
    # Quitar los ejes
    ax.set_axis_off()

    # Añadir título
    plt.title(f"Tamaño de los clusters para la fecha: {fecha_str}")

    # Mostrar el mapa
    plt.show()

# Usar interact para actualizar dinámicamente
widgets.interactive(agregar, fecha_str=fecha_selector)

A continuación, generamos el dendograma para ver si hemos seleccionado un número correcto de clusters. Como podemos observar, parece que el número más idóneo independientemente de la fecha son 2 clústers (en algunos casos 3). La fecha que estamos tomando como ejemplo no es una excepción y aparentemente tiene un número ideal de 2 clusters

In [ ]:
# Crear un widget de selección de fecha
fecha_selector = widgets.Dropdown(
    options=fechas_unicas,
    description="Fecha:",
    style={'description_width': 'initial'}
)

# Función para actualizar el mapa según la fecha seleccionada
def dendro(fecha_str):
  # Convertir la fecha seleccionada de string a datetime
    fecha = pd.to_datetime(fecha_str, format='%Y/%m/%d')

    # Filtrar el DataFrame por la fecha seleccionada
    df_filtrado = df_union[df_union['Fecha'] == fecha]

    # Filtrar por delitos
    data = df_filtrado[delitos].values

    # Calcular la matriz de enlace usando el método 'ward'
    Z = linkage(data, method='ward')

    # Crear el dendrograma truncado mostrando sólo los últimos 10 clusters
    plt.figure(figsize=(12, 6))
    dendrogram(Z, truncate_mode='lastp', p=13, leaf_rotation=90, leaf_font_size=10)
    plt.title("Dendrograma (truncado)")
    plt.xlabel("Índice del cluster o muestra")
    plt.ylabel("Distancia")
    plt.show()

# Usar interact para actualizar dinámicamente
widgets.interactive(dendro, fecha_str=fecha_selector)

Aparte de utilizar un dendograma, también existen otros métodos para determinar el número ideal de clústers. En primer lugar tenemos el índice de silueta, que es una medida de evaluación utilizada para determinar la calidad de un agrupamiento (clustering). Se usa para evaluar cuán bien se ha realizado el agrupamiento comparando la distancia dentro de los clusters y la distancia entre los clusters. Su valor ayuda a entender qué tan coherentes y bien separados están los grupos formados. Si valores son cercanos a 1 los puntos están bien agrupados, y los clusters son compactos y bien separados. Si el valor es cercano a 0 los puntos están en la frontera entre clusters, lo que indica que no están claramente asignados a uno u otro cluster. Si el valor es cercano a -1 los puntos están mal asignados, ya que deberían pertenecer a otro cluster.

Con el siguiente código se puede seleccionar el número óptimo de casos según el índice de silueta para cada fecha, que en la mayoría de los casos es 2.

In [ ]:
# Crear un widget de selección de fecha
fecha_selector = widgets.Dropdown(
    options=fechas_unicas,
    description="Fecha:",
    style={'description_width': 'initial'}
)

def indice_silueta_optimo(fecha_str):
    # Convertir la fecha seleccionada de string a datetime
    fecha = pd.to_datetime(fecha_str, format='%Y/%m/%d')

    # Filtrar el DataFrame por la fecha seleccionada
    df_filtrado = df_union[df_union['Fecha'] == fecha]

    # Convertir los datos a un array NumPy
    data = df_filtrado[delitos].values

    mejor_silueta = -1  # Valor inicial para el índice de silueta
    mejor_clusters = 2  # Número inicial de clusters

    # Probar varios números de clusters, por ejemplo, de 2 a 10
    for n_clusters in range(2, 11):
        model = AgglomerativeClustering(n_clusters=n_clusters)
        labels = model.fit_predict(data)  # Obtener etiquetas de los clusters
        silhouette_avg = silhouette_score(data, labels)  # Calcular índice de silueta

        print(f"Índice de silueta para {n_clusters} clusters: {silhouette_avg:.4f}")

        # Si encontramos un índice de silueta mayor, actualizamos el mejor número de clusters
        if silhouette_avg > mejor_silueta:
            mejor_silueta = silhouette_avg
            mejor_clusters = n_clusters

    print(f"\nEl mejor número de clusters es: {mejor_clusters} con un índice de silueta de {mejor_silueta:.4f}")

# Crear un selector de fecha
widgets.interactive(indice_silueta_optimo, fecha_str=fecha_selector)


Otra forma de determinar el mejor número de clusters es el método del codo que es una técnica muy utilizada para determinar el número óptimo de clusters (k) en algoritmos de clustering. Se basa en graficar el error cuadrático dentro del cluster en funciónn de los diferentes valores de k.El gráfico generalmente muestra una disminución rápida de WCSS cuando se aumenta k al principio, y luego la disminución se hace más gradual. El "codo" es el punto donde esta disminución empieza a ser más lenta. El número de clusters en este punto (el "codo") es el número óptimo de clusters, ya que después de este punto, añadir más clusters no reduce significativamente la dispersión dentro de los clusters.

Si observamos los resultados obtenidos mediante este método podemos ver como coincide con el indice de silueta, ya que en la mayoría de fechas el mejor numero de clusters es 2.

Tanto en con el índice de silueta como con el método del codo el número óptimo de clusters para la fecha que tomamos como ejemplo es 2.

In [ ]:
# Crear un widget de selección de fecha
fecha_selector = widgets.Dropdown(
    options=fechas_unicas,
    description="Fecha:",
    style={'description_width': 'initial'}
)

def metodo_del_codo(fecha_str):
    # Convertir la fecha seleccionada de string a datetime
    fecha = pd.to_datetime(fecha_str, format='%Y/%m/%d')

    # Filtrar el DataFrame por la fecha seleccionada
    df_filtrado = df_union[df_union['Fecha'] == fecha]

    # Convertir los datos a un array NumPy
    data = df_filtrado[delitos].values

    # Probar diferentes números de clusters, por ejemplo, de 1 a 10
    sse = []
    for n_clusters in range(1, 11):
        model = KMeans(n_clusters=n_clusters)
        model.fit(data)
        sse.append(model.inertia_)  # Guardar el SSE para cada número de clusters

    # Graficar el método del codo
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, 11), sse, marker='o')
    plt.title('Método del Codo')
    plt.xlabel('Número de Clusters')
    plt.ylabel('SSE')
    plt.show()

# Crear un selector de fecha
widgets.interactive(metodo_del_codo, fecha_str=fecha_selector)


Una vez que hemos seleccionado la manera de determinar el mejor número de clusters, lo que vamos a hacer es agrupar según ese número de clusters y disolver los distritos que pertenezcan al mismo grupo mediante la función dissolve, definida a continuación.

In [ ]:
def dissolve(gs):
    '''
    Toma una serie de polígonos y los une en un solo polígono

    Argumentos
    ---------
    gs        : GeoSeries
                Secuencia de polígonos a disolver
    Devuelve
    -------
    disuelto : Polígono
               Un único polígono que contiene todos los polígonos en `gs`
    '''
    return gs.union_all()

A continuación se puede observar el resultado en un mapa de agrupar por el mejor número de clusters.

In [ ]:
# Crear un widget de selección de fecha
fecha_selector = widgets.Dropdown(
    options=fechas_unicas,
    description="Fecha:",
    style={'description_width': 'initial'}
)

def cambio_2(fecha_str):
    # Convertir la fecha seleccionada de string a datetime
    fecha = pd.to_datetime(fecha_str, format='%Y/%m/%d')

    # Filtrar el DataFrame por la fecha seleccionada
    df_filtrado = df_union[df_union['Fecha'] == fecha]

    # Convertir los datos a un array NumPy
    data = df_filtrado[delitos].values

    # Variables para determinar el mejor número de clusters (con índice de silueta)
    mejor_silueta = -1
    mejor_clusters = 2

    # Probar varios números de clusters (por ejemplo, de 2 a 10)
    for n_clusters in range(2, 11):
        model = AgglomerativeClustering(n_clusters=n_clusters)
        labels = model.fit_predict(data)  # Obtener etiquetas de los clusters
        silhouette_avg = silhouette_score(data, labels)  # Calcular índice de silueta

        # Actualizar el mejor número de clusters basado en el índice de silueta
        if silhouette_avg > mejor_silueta:
            mejor_silueta = silhouette_avg
            mejor_clusters = n_clusters

    # Ahora usa el mejor número de clusters encontrado
    s_agg13 = AgglomerativeClustering(n_clusters=mejor_clusters, connectivity=w.sparse)

    # Ejecutar el algoritmo de clustering aglomerativo
    s_agg13clust = s_agg13.fit(df_filtrado[delitos])
    df_filtrado['sagg13clust'] = s_agg13clust.labels_

    # Disolver las geometrías basadas en los clusters
    distritos_delitos = gpd.GeoSeries(
        df_filtrado.groupby(df_filtrado['sagg13clust']).apply(dissolve),
        crs=df_filtrado.crs
    )

    # Configurar la figura y el eje para la visualización
    f, ax = plt.subplots(1, figsize=(6, 6))

    # Dibujar el mapa base de OpenStreetMap con contextily
    distritos_delitos = distritos_delitos.to_crs(epsg=3857)  # Cambiar a coordenadas web Mercator
    distritos_delitos.plot(ax=ax, linewidth=0.5, facecolor='white', edgecolor='k', alpha=0.8)  # alpha para opacidad

    # Añadir el mapa base
    cx.add_basemap(ax, crs=distritos_delitos.crs.to_string(), source=cx.providers.OpenStreetMap.Mapnik)

    # Eliminar el eje
    ax.set_axis_off()

    # Añadir el título
    plt.title(f'Distritos basados en delitos para {mejor_clusters} clusters')
    plt.show()

# Crear un selector de fecha
widgets.interactive(cambio_2, fecha_str=fecha_selector)

Por último, podemos ver la diferencia entre la distribución de los distritos real (en azul) y la generada según los delitos (naranja). Para la mayoría de las fechas el resultado es el mismo, generando un grupo para el distrito centro y otro para el resto de los distritos. En algunas fechas, como noviembre de 2021 se generan tres grupos según delito, pero hay algo en común y es que en todos los casos parece haber una diferencia clara entre el centro de la ciudad y el resto de los distritos. Para saber las causas de esta distribución de delitos podría ser interesante incluir más variables, como centros de ocio, números de de comisarías o patrullas de policía, afluencia de turistas.

In [ ]:
# Crear un widget de selección de fecha
fecha_selector = widgets.Dropdown(
    options=fechas_unicas,
    description="Fecha:",
    style={'description_width': 'initial'}
)

def cambio_3(fecha_str):
    # Convertir la fecha seleccionada de string a datetime
    fecha = pd.to_datetime(fecha_str, format='%Y/%m/%d')

    # Filtrar el DataFrame por la fecha seleccionada
    df_filtrado = df_union[df_union['Fecha'] == fecha]

    # Convertir los datos a un array NumPy
    data = df_filtrado[delitos].values

    # Variables para determinar el mejor número de clusters (con índice de silueta)
    mejor_silueta = -1
    mejor_clusters = 2

    # Probar varios números de clusters (por ejemplo, de 2 a 10)
    for n_clusters in range(2, 11):
        model = AgglomerativeClustering(n_clusters=n_clusters)
        labels = model.fit_predict(data)  # Obtener etiquetas de los clusters
        silhouette_avg = silhouette_score(data, labels)  # Calcular índice de silueta

        # Actualizar el mejor número de clusters basado en el índice de silueta
        if silhouette_avg > mejor_silueta:
            mejor_silueta = silhouette_avg
            mejor_clusters = n_clusters

    # Ahora usa el mejor número de clusters encontrado
    s_agg13 = AgglomerativeClustering(n_clusters=mejor_clusters, connectivity=w.sparse)

    # Ejecutar el algoritmo de clustering aglomerativo
    s_agg13clust = s_agg13.fit(df_filtrado[delitos])
    df_filtrado['sagg13clust'] = s_agg13clust.labels_

    # Disolver las geometrías basadas en los clusters
    distritos_delitos = gpd.GeoSeries(
        df_filtrado.groupby(df_filtrado['sagg13clust']).apply(dissolve),
        crs=df_filtrado.crs
    )
    # Crear la imagen
    f, ax = plt.subplots(1, figsize=(12, 12))
    f.set_facecolor("w")

    # Añadir los distritos
    df_filtrado.to_crs(
        epsg=25830
    ).plot(
        ax=ax,
        facecolor="none",
        edgecolor="xkcd:salmon",
        linewidth=2
    )

    # Añadir la regionalización
    distritos_delitos.to_crs(
        epsg=25830
    ).plot(
        ax=ax,
        facecolor="none",
        edgecolor="xkcd:blue",
        linewidth=1.5
    )

    # Añadir el mapa base
    cx.add_basemap(
        ax,
        crs="EPSG:25830",
        source=cx.providers.CartoDB.PositronNoLabels
    )

    # Quitar ejes
    ax.set_axis_off()

    # Añadir el título
    plt.title(f'Distritos reales y regionalización')

    # Mostrar la imagen
    plt.show()

# Crear un selector de fecha
widgets.interactive(cambio_3, fecha_str=fecha_selector)

**ANÁLISIS DE PATRONES DE PUNTOS APLICADO A MULTAS DE TRÁFICO EN MADRID**

**Visualización de un patrón de puntos**

Como estos datos no se exploraron en el EDA anterior, realizamos una inspección visual de los mismos a través de varios mapas.

In [ ]:
# Plotear puntos
gdf_multas.plot.scatter("LONGITUD", "LATITUD")

In [ ]:
# Plotear puntos
ax = gdf_multas.plot.scatter(
    "LONGITUD",
    "LATITUD",
    s=0.25,
    c="xkcd:bright yellow",
    alpha=0.5,
    figsize=(9, 9)
)

# Eliminar ejes
ax.set_axis_off()

# Añadir mapa base con tonalidad oscura
cx.add_basemap(
    ax,
    crs="EPSG:25830",
    source=cx.providers.CartoDB.DarkMatter
)

In [ ]:
# Plotear con la distribución de puntos por latitud y longitud
joint_axes = sns.jointplot(
    x='LONGITUD', y='LATITUD', data=gdf_multas, s=1
)

# Añadir mapa base
cx.add_basemap(
    joint_axes.ax_joint,
    crs="EPSG:25830",
    source=cx.providers.CartoDB.PositronNoLabels
)

**Unión de puntos en polígonos**

En este punto, se propone visualizar los datos puntuales en forma de polígonos, convirtiéndolos para poder mapear las multas en forma de coropletas. En primer lugar, se representarán como rejillas irregulares con la delimitiación de los distritos de Madrid.

In [ ]:
# Convertir los distritos a 25830 para visualizarlo correctamente
distritos = distritos.to_crs(epsg=25830)

# Plotear puntos
ax = gdf_multas.plot.scatter(
    "LONGITUD",
    "LATITUD",
    s=0.25,
    c="xkcd:bright yellow",
    alpha=0.5,
    figsize=(9, 9)
)

# Añadir delimitación de distritos
distritos.plot(
    ax=ax,
    facecolor="none",
    edgecolor="xkcd:pale lavender"
)

# Eliminar ejes
ax.set_axis_off()

# Añadir mapa base con tonalidad oscura
cx.add_basemap(
    ax,
    crs="EPSG:25830",
    source=cx.providers.CartoDB.DarkMatter
)

Ahora obtendremos cuántas multas hay en cada uno de los distritos de Madrid. Para ello agruparemos los datos por la columna "DISTRITO" y aplicaremos el método size para contar las multas de cada distrito, organizando los datos en la columna "multas_zona" de la nueva tabla "multas_distritos".

In [ ]:
# Contar multas por distritos
multas_distritos = gdf_multas.groupby("DISTRITO").size()

# Asignar el número de multas por distrito a la columna "multas_zona" y unir por distritos
areas = distritos.join(
    pd.DataFrame({"multas_zona": multas_distritos}),
    on="DISTRITO"
)

areas.head()

In [ ]:
# Establecer figura y ejes
f, ax = plt.subplots(1, figsize=(9, 9))

# Plotear multas por distrito por el métodos de cuantiles y con transparencia
areas.plot(
    column='multas_zona',
    scheme='quantiles',
    ax=ax,
    legend=True,
    legend_kwds={"loc": 4},
    alpha=0.4
)

# Eliminar los ejes
ax.set_axis_off()

# Título del gráfico
ax.set_title("Multas por distritos de Madrid")

# Añadir mapa base
cx.add_basemap(
    ax,
    crs="EPSG:25830",
    source=cx.providers.CartoDB.DarkMatterNoLabels
)

# Mapear
plt.show()

**Densidad de multas por distrito**

También resulta útil conocer la densidad de multas por área, en este caso por kilómetros cuadrados. Para ello, una vez tenemos el área de cada distrito, hacemos el conteo de multas en base al área de cada distrito.

In [ ]:
# Obtener el área (en kilómetros cuadrados) de cada distrito
areas["area_km2"] = areas.to_crs(epsg=25830).area * 1e-6
areas.head()

In [ ]:
areas["densidad_multas"] = areas["multas_zona"] / areas["area_km2"]

In [ ]:
# Establecer figura y ejes
f, ax = plt.subplots(1, figsize=(9, 9))

# Plotear densidad de multas por distrito por el métodos de cuantiles y con transparencia
areas.plot(
    column='densidad_multas',
    scheme='quantiles',
    ax=ax,
    legend=True,
    legend_kwds={"loc": 4},
    alpha=0.4
)

# Eliminar ejes
ax.set_axis_off()

# Título
ax.set_title("Densidad de multas por km2 en los distritos de Madrid")

# Añadir mapa base con tonalidad oscura
cx.add_basemap(
    ax,
    crs="EPSG:25830",
    source=cx.providers.CartoDB.DarkMatterNoLabels
)

# Mapear
plt.show()

Evidentemente, los distritos con mayor densidad de multas son las áreas céntricas y más concurridas, como el distrito Centro, Chamberí y Chamartín.

También se puede convertir estos datos puntuales en forma de polígonos, pero en este caso como rejillas regulares, en forma de una cuadrícula de hexágonos, por ejemplo, lo cual tiene la ventaja de proporcionar una topología regular en la que cada polígono tiene el mismo tamaño y forma.

In [ ]:
# Establecer figura y ejes
f, ax = plt.subplots(1, figsize=(16, 12))

# Añadir los datos puntuales de multas en forma de hexágonos
hb = ax.hexbin(
    gdf_multas["LONGITUD"],
    gdf_multas["LATITUD"],
    gridsize=50,
    alpha=0.5,
    edgecolor="none"
)

# Añadir barra de color
plt.colorbar(hb)

# Eliminar ejes
ax.set_axis_off()

# Añadir mapa base
cx.add_basemap(
    ax,
    crs="EPSG:25830",
    source=cx.providers.CartoDB.DarkMatterNoLabels
)

# Título
ax.set_title("Hex-binning de multas en Madrid")

# Mepear
plt.show()

Gracias a este mapa podemos observar que el punto en el que mayor cantidad de multas hubo en Madrid en mayo de 2024 se ubica en un parking junto al recinto de la Caja Mágica, donde durante este mes tuvieron lugar importantes eventos como el Mutua Madrid Open de tenis o el Festival Tomavistas Madrid, que tuvieron una gran afluencia de público y por tanto dispararon la cifra de multas a su alrededor.

**Estimación de la densidad de kernel**

Para contar el número de multas en Madrid de manera continua en el espacio, emplearemos el método de estimación de densidad de kernel (KDE), en base a una muestra de 1.000 puntos aleatorios de los datos originales.

In [ ]:
# Tomar una muestra de 1.000 puntos de manera aleatoria
gdf_multas_sub = gdf_multas.sample(1000, random_state=12345)

In [ ]:
sns.kdeplot(
    x="LONGITUD",
    y="LATITUD",
    data=gdf_multas_sub,
    n_levels=50,
    fill=True,
    cmap='BuPu'
)

In [ ]:
# Establecer figura y ejes
f, ax = plt.subplots(1, figsize=(12, 12))

# Añadir el cálculo de la densidad de kernel
sns.kdeplot(
    x='LONGITUD',
    y='LATITUD',
    data=gdf_multas_sub,
    n_levels=50,
    fill=True,
    alpha=0.25,
    cmap="viridis"
)

# Eliminar ejes
ax.set_axis_off()

# Añadir mapa base
cx.add_basemap(
    ax,
    crs="EPSG:25830",
    source=cx.providers.CartoDB.DarkMatterNoLabels
)

# Título
ax.set_title("KDE de multas en Madrid")

# Mapear
plt.show()

Gráfico KDE bivariado con contornos

In [ ]:
joint_axes = sns.jointplot(
    x='LONGITUD', y='LATITUD', data=gdf_multas_sub, kind="kde"
)

cx.add_basemap(
    joint_axes.ax_joint,
    crs="EPSG:25830",
    source=cx.providers.CartoDB.PositronNoLabels
)

**Análisis estadístico de patrones de puntos con funciones de Ripley**

In [ ]:
coordinates = gdf_multas_sub[['LONGITUD','LATITUD']].values
coordinates

**G de Ripley**

In [ ]:
g_test = distance_statistics.g_test(
    coordinates, support=40, keep_simulations=True, n_simulations=999
)

In [ ]:
# Establecer figura y ejes
f, ax = plt.subplots(1, 2, figsize=(9, 3), gridspec_kw=dict(width_ratios=(6, 3)))

# Graficar todas las simulaciones
ax[0].plot(g_test.support, g_test.simulations.T, color='k', alpha=.01)

# Mostrar la mediana de las simulaciones
ax[0].plot(g_test.support, np.median(g_test.simulations, axis=0), color='cyan',
           label='mediana de simulaciones')

# Graficar la función G observada del patrón
ax[0].plot(g_test.support, g_test.statistic, label='observado', color='red')

# Limpiar etiquetas y ejes
ax[0].set_xlabel('distancia')
ax[0].set_ylabel('% de distancias al vecino más cercano\nmenores')
ax[0].legend()
ax[0].set_xlim(0, 2000)
ax[0].set_title(r"Función $G(d)$ de Ripley")

# Graficar el patrón de puntos en el segundo subgráfico
ax[1].scatter(*coordinates.T)

# Limpiar etiquetas y ejes en el segundo subgráfico
ax[1].set_xticks([])
ax[1].set_yticks([])
ax[1].set_xticklabels([])
ax[1].set_yticklabels([])
ax[1].set_title('Patrón de puntos')
f.tight_layout()
plt.show()

En el gráfico se observa que la función observada aumenta de manera muy rápida en comparación con los patrones simulados, por lo que el patrón observado sí está agrupado, ya que sus puntos están más cerca entre sí de lo que sería esperado en una distribución aleatoria.

**F de Ripley**

In [ ]:
f_test = distance_statistics.f_test(
    coordinates, support=40, keep_simulations=True, n_simulations=999
)

In [ ]:
# Establecer figura y ejes
f, ax = plt.subplots(1, 2, figsize=(9, 3), gridspec_kw=dict(width_ratios=(6, 3)))

# Graficar todas las simulaciones
ax[0].plot(f_test.support, f_test.simulations.T, color='k', alpha=.01)

# Mostrar la mediana de las simulaciones
ax[0].plot(f_test.support, np.median(f_test.simulations, axis=0), color='cyan',
           label='mediana de simulaciones')

# Graficar la función F observada del patrón
ax[0].plot(f_test.support, f_test.statistic, label='observado', color='red')

# Limpiar etiquetas y ejes
ax[0].set_xlabel('distancia')
ax[0].set_ylabel('% de distancias menores\nal punto más cercano en el patrón')
ax[0].legend()
ax[0].set_xlim(0, 2000)
ax[0].set_title(r"Función $F(d)$ de Ripley")

# Graficar el patrón de puntos en el segundo subgráfico
ax[1].scatter(*coordinates.T)

# Limpiar etiquetas y ejes en el segundo subgráfico
ax[1].set_xticks([])
ax[1].set_yticks([])
ax[1].set_xticklabels([])
ax[1].set_yticklabels([])
ax[1].set_title('Patrón de puntos')
f.tight_layout()
plt.show()

En este caso, la función observada aumenta más lentamente en comparación con los patrones simulados, por lo que el patrón observado sí está agrupado, ya que la cantidad de grandes áreas vacías es superior en el patrón de la simulaciones.

**Agrupaciones o clusters de puntos**

In [ ]:
# Configurar el algoritmo DBSCAN
algorit = DBSCAN(eps=100, min_samples=50) # Mínimo 50 multas dentro de un radio de 100 metros

In [ ]:
# Ajustar el cluster a los datos de multas
algorit.fit(gdf_multas[["LONGITUD", "LATITUD"]])

In [ ]:
# Obtener las etiquetas
algorit.labels_

In [ ]:
# Mostrar los primeros 5 valores
algorit.core_sample_indices_[:5]

In [ ]:
# Convertir las etiquetas en un objeto Serie
labels = pd.Series(algorit.labels_, index=gdf_multas.index)
labels

In [ ]:
# Establecer figura y ejes
f, ax = plt.subplots(1, figsize=(6, 6))

# Asignar etiquetas a la tabla de multas y seleccionar los puntos que no forman parte de ningún clúster (ruido)
ruido = gdf_multas.assign(
    labels=labels
).query("labels == -1")

# Representar el ruido en color gris
ax.scatter(
    ruido["LONGITUD"],
    ruido["LATITUD"],
    c='grey',
    s=5,
    linewidth=0
)

# Representar todos los puntos que no son ruido en rojo
ax.scatter(
    gdf_multas.loc[gdf_multas.index.difference(ruido.index), "LONGITUD"],
    gdf_multas.loc[gdf_multas.index.difference(ruido.index), "LATITUD"],
    c="red",
    linewidth=0
)

# Añadir mapa base
cx.add_basemap(
    ax,
    crs="EPSG:25830",
    source=cx.providers.CartoDB.PositronNoLabels
)

# Mapear
plt.show()

In [ ]:
# Configurar el algoritmo DBSCAN
algo = DBSCAN(eps=500, min_samples=10)  # Mínimo 10 multas dentro de un radio de 500 metros

# Ajustar a los puntos de las multas
algorit.fit(gdf_multas[["LONGITUD", "LATITUD"]])

# Almacenar etiquetas
labels = pd.Series(algorit.labels_, index=gdf_multas.index)

In [ ]:
# Establecer figura y ejes
f, ax = plt.subplots(1, figsize=(6, 6))

# Asignar etiquetas a la tabla de multas y seleccionar los puntos que no forman parte de ningún clúster (ruido)
noise = gdf_multas.assign(
    labels=labels
).query("labels == -1")

# Representar el ruido en color gris
ax.scatter(
    noise["LONGITUD"],
    noise["LATITUD"],
    c='grey',
    s=5,
    linewidth=0
)
# Representar todos los puntos que no son ruido en rojo
ax.scatter(
    gdf_multas.loc[gdf_multas.index.difference(noise.index), "LONGITUD"],
    gdf_multas.loc[gdf_multas.index.difference(noise.index), "LATITUD"],
    c="red",
    linewidth=0
)

# Añadir mapa base
cx.add_basemap(
    ax,
    crs="EPSG:25830",
    source=cx.providers.CartoDB.PositronNoLabels
)

# Mapear
plt.show()

In [ ]:
def clusters(db, eps, min_samples):
    '''
    Calcula y visualiza los clústeres de DBSCAN
    ...

    Argumentos
    ----------
    db          : (Geo)DataFrame
                  Tabla con al menos las columnas `X` y `Y` para las coordenadas de los puntos
    eps         : float
                  Radio máximo para buscar puntos dentro de un clúster
    min_samples : int
                  Número mínimo de puntos en un clúster
    '''
    # Congigurar algoritmo
    algo = DBSCAN(eps=eps, min_samples=min_samples)
    algo.fit(db[['LONGITUD', 'LATITUD']])
    lbls = pd.Series(algo.labels_, index=db.index)

    # Representar todos los puntos según si son ruido o no ruido
    f, ax = plt.subplots(1, figsize=(6, 6))
    noise = db.loc[lbls==-1, ['LONGITUD', 'LATITUD']]
    ax.scatter(noise['LONGITUD'], noise['LATITUD'], c='grey', s=5, linewidth=0)
    ax.scatter(
        db.loc[db.index.difference(noise.index), 'LONGITUD'],
        db.loc[db.index.difference(noise.index), 'LATITUD'],
        c='red',
        linewidth=0
    )

    # Añadir mapa base
    cx.add_basemap(
    ax,
    crs="EPSG:25830",
    source=cx.providers.CartoDB.PositronNoLabels)

    # Mapear
    return plt.show()

In [ ]:
clusters(gdf_multas, 50, 100)

In [ ]:
interact(
    clusters,                 # Método para hacerlo interactivo
    db=fixed(gdf_multas),     # Datos para pasar a db
    eps=(50, 500, 50),        # Rango de inicio, final y paso de distancia
    min_samples=(50, 300, 50) # Rango inicio, final y paso de muestras mínimas
)

Visualizar gráfica del codo para elegir ϵ óptimo de manera visual

In [ ]:
# Definir el número de vecinos (minPts - 1)
minPts = 50  # comenzamos con 50
k = minPts - 1

# Ajustar el modelo a los datos de multas
neigh = NearestNeighbors(n_neighbors=k)
neigh.fit(gdf_multas[['LONGITUD', 'LATITUD']])

# Calcular distancias al k-ésimo vecino más cercano
distances, _ = neigh.kneighbors(gdf_multas[['LONGITUD', 'LATITUD']])
k_distances = np.sort(distances[:, k-1])[::-1]  # Ordenar de mayor a menor

# Filtrar distancias mayores a 0
k_distances = k_distances[k_distances > 0]

# Visualizar la curva del codo en el orden correcto
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(k_distances) + 1), k_distances, marker='.', linestyle='-', label="Distancias ordenadas")

# Configuración del gráfico
plt.xlabel('Índice de puntos ordenados')
plt.ylabel(f'Distancia al {k}-ésimo vecino más cercano')
plt.title('Método del Codo para determinar $\\epsilon$ en DBSCAN')
plt.legend()
plt.grid(True)

# Ajustar los ticks en el eje X para mayor claridad
plt.xticks(np.arange(0, len(k_distances) + 1, step=200))

# Mostrar gráfico
plt.show()

In [ ]:
# Cargar datos de multas en Madrid
data = gdf_multas[['LONGITUD', 'LATITUD']].values

# Definir valores de epsilon y minPts
eps_values = [100, 500, 5]
minPts_values = [400, 700, 1000]

fig, axes = plt.subplots(len(minPts_values), len(eps_values), figsize=(15, 8))

for i, minPts in enumerate(minPts_values):
    for j, eps in enumerate(eps_values):
        db = DBSCAN(eps=eps, min_samples=minPts).fit(data)
        labels = db.labels_
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)  # No cuenta ruido (-1)

        # Usar colores de la paleta "tab10" con color negro para ruido
        unique_labels = set(labels)
        colors = plt.cm.tab10(np.linspace(0, 1, len(unique_labels)))
        color_dict = {label: color for label, color in zip(unique_labels, colors)}
        color_dict[-1] = 'black'  # Ruido en negro

        # Aplicar colores personalizados
        cluster_colors = [color_dict[label] for label in labels]

        # Graficar puntos coloreados por clúster
        axes[i, j].scatter(data[:, 0], data[:, 1], c=cluster_colors, s=5, alpha=0.6, edgecolors='k', linewidth=0.1)
        axes[i, j].set_title(f"ε={eps:.2f}, minPts={minPts}\nClústeres={n_clusters}", fontsize=9, pad=8)
        axes[i, j].set_xticks([])
        axes[i, j].set_yticks([])

# Ajustar espacio entre gráficos para evitar solapamientos
plt.subplots_adjust(hspace=0.4, wspace=0.3)

# Mostrar gráfico
plt.show()

Índice silhouette para comprobar la bondad de la clusterización

In [ ]:
# Excluir ruido (-1) de DBSCAN antes de calcular el índice: con epsilon de 170 y con minPts de 70
mask = labels != -1
score = silhouette_score(gdf_multas[['LONGITUD', 'LATITUD']][mask], labels[mask])
print(f"Índice de Silhouette: {score:.3f}")

Al ser valor 1, las observaciones dentro de cada clúster están perfectamente agrupadas y cada grupo está correctamente separado del resto.

**Anexo. Uso de algoritmos de clasificación.**


Una vez explorados los datos desde distintos puntos de vista, una posibilidad interesante es clasificar los datos. Para ello existen diversos algoritmos que se pueden utilizar como las redes neuronales, las máquinas vector soporte (SVM), los árboles de decisión o random forest. Para generar estas clasificaciones hemos utilizado la documentación oficial de [scikit-learn](https://scikit-learn.org/stable/supervised_learning.html).

En primer lugar vamos a probar a realizar una clasificación mediante árboles de decisión. Para ello tomamos como variables predictoras el distrito, año, mes y número de delitos (serán las mismas variables predictoras en el resto de algoritmos) y como variable objetivo o variable a predecir es el riesgo que presenta la zona, que según el número de delitos que ha presentado una zona respecto a la media se considera de Alto riesgo (33% de los valores más altos), Bajo riesgo (33% de los valores mas bajos) y Riesgo Medio (resto de valores).

Según el tipo de delito, la clasificación cambia, indicando que hay zonas muy seguras en general, otras que son peligras para todo tipo de delitos y algunas en las que ciertos tipos de delitos son predominantes.

In [ ]:
# Codificar los distritos como números
le = LabelEncoder()
df_union["distrito_cod"] = le.fit_transform(df_union["DISTRITOS"])
# Añadir variables de mes y año para añadirlas como variables
df_union["año"] = df_union["Fecha"].dt.year
df_union["mes"] = df_union["Fecha"].dt.month

In [ ]:
# Función de clasificación
def clasificar_delito(tipo_delito):

    # Calcular percentiles para el tipo de delito seleccionado
    percentil_bajo = np.percentile(df_union[tipo_delito], 33)  # 33% de los valores más bajos
    percentil_alto = np.percentile(df_union[tipo_delito], 66)  # 33% de los valores más altos

    # Clasificar los distritos en base a los percentiles de delitos
    def clasificar_distrito(row):
        if row[tipo_delito] > percentil_alto:
            return 'Alto riesgo'
        elif row[tipo_delito] > percentil_bajo:
            return 'Riesgo medio'
        else:
            return 'Bajo riesgo'

    # Crear la variable objetivo clasificando los distritos
    df_union['clasificacion'] = df_union.apply(clasificar_distrito, axis=1)

    # Preprocesar la variable objetivo utilizando LabelEncoder
    le = LabelEncoder()
    y = df_union['clasificacion']
    y = le.fit_transform(y)  # Codificar categorías a números

    # Definir las variables predictoras (X)
    X = df_union[["distrito_cod", "año", "mes", tipo_delito]]

    # Dividir los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entrenar el modelo de clasificación
    modelo = DecisionTreeClassifier(max_depth=5, random_state=42)
    modelo.fit(X_train, y_train)

    # Hacer predicciones en todo el conjunto de datos
    df_union["prediccion"] = modelo.predict(X)

    # Invertir la transformación del LabelEncoder para obtener las etiquetas originales
    df_union["prediccion"] = le.inverse_transform(df_union["prediccion"])

    # Graficar el mapa
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    df_union.plot(column="prediccion", cmap="tab10", legend=True, ax=ax, edgecolor="black")

    # Agregar nombres de los distritos en el mapa
    for x, y, label in zip(df_union.geometry.centroid.x, df_union.geometry.centroid.y, df_union["DISTRITOS"]):
        ax.text(x, y, label, fontsize=8, ha="center", color="black")

    ax.set_title(f"Mapa de clasificación por árboles de decisión para {tipo_delito}", fontsize=14)
    ax.axis("off")  # Ocultar ejes

    plt.show()

# Crear el widget interactivo para seleccionar el tipo de delito
interact(clasificar_delito, tipo_delito=widgets.Dropdown(options=delitos, description="Delito:"));


A continuación, realizamos la misma clasificación con otro algoritmo, el random forest. Este algoritmo es muy utilizado en muchos campos y es más robusto que los árboles de decisión. Sin embargo, cuenta con una desventaja respecto a los árboles de decisión, y es que no se pueden ver los nodos. Sin embargo, en el ejemplo utilizando árbol de decisión no se han mostrado los nodos y ramas del árbol porque había muchos y la visualización era bastante mala.

Dicho esto, los resultados con random forest son bastante similares a los obtenidos mediante árboles de decisión.

In [ ]:
# Definir la función de clasificación
def clasificar_delito(tipo_delito):

    # Calcular percentiles para el tipo de delito seleccionado
    percentil_bajo = np.percentile(df_union[tipo_delito], 33)  # 33% de los valores más bajos
    percentil_alto = np.percentile(df_union[tipo_delito], 66)  # 33% de los valores más altos

    # Clasificar los distritos en base a los percentiles de delitos
    def clasificar_distrito(row):
        if row[tipo_delito] > percentil_alto:
            return 'Alto riesgo'
        elif row[tipo_delito] > percentil_bajo:
            return 'Riesgo medio'
        else:
            return 'Bajo riesgo'

    # Crear la variable objetivo clasificando los distritos
    df_union['clasificacion'] = df_union.apply(clasificar_distrito, axis=1)

    # Preprocesar la variable objetivo utilizando LabelEncoder
    le = LabelEncoder()
    y = df_union['clasificacion']
    y = le.fit_transform(y)  # Codificar categorías a números

    # Definir las variables predictoras (X)
    X = df_union[["distrito_cod", "año", "mes", tipo_delito]]

    # Dividir los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entrenar el modelo de Random Forest
    modelo = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
    modelo.fit(X_train, y_train)

    # Hacer predicciones en todo el conjunto de datos
    df_union["prediccion"] = modelo.predict(X)

    # Invertir la transformación del LabelEncoder para obtener las etiquetas originales
    df_union["prediccion"] = le.inverse_transform(df_union["prediccion"])

    # Graficar el mapa
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    df_union.plot(column="prediccion", cmap="tab10", legend=True, ax=ax, edgecolor="black")

    # Agregar nombres de los distritos en el mapa
    for x, y, label in zip(df_union.geometry.centroid.x, df_union.geometry.centroid.y, df_union["DISTRITOS"]):
        ax.text(x, y, label, fontsize=8, ha="center", color="black")

    ax.set_title(f"Mapa de clasificación con random forest para {tipo_delito}", fontsize=14)
    ax.axis("off")  # Ocultar ejes

    plt.show()

# Crear el widget interactivo para seleccionar el tipo de delito
interact(clasificar_delito, tipo_delito=widgets.Dropdown(options=delitos, description="Delito:"));


Después, hacemos una prueba con el algoritmo de redes neuronales. A diferencia de los casos anteriores, los resultados han variado más hasta el punto de que si nos fijamos en la clasificación . Es importante tener en cuenta que para este algoritmo hay que ajustar bien algunos parámetros como el número de capas de neuronas, el número de neuronas o la tasa de crecimiento para obtener los mejores resultados y evitar problemas como el sobreajuste.

In [ ]:
# Definir la función de clasificación
def clasificar_delito(tipo_delito):

    # Calcular percentiles para el tipo de delito seleccionado
    percentil_bajo = np.percentile(df_union[tipo_delito], 33)  # 33% de los valores más bajos
    percentil_alto = np.percentile(df_union[tipo_delito], 66)  # 33% de los valores más altos

    # Clasificar los distritos en base a los percentiles de delitos
    def clasificar_distrito(row):
        if row[tipo_delito] > percentil_alto:
            return 'Alto riesgo'
        elif row[tipo_delito] > percentil_bajo:
            return 'Riesgo medio'
        else:
            return 'Bajo riesgo'

    # Crear la variable objetivo clasificando los distritos
    df_union['clasificacion'] = df_union.apply(clasificar_distrito, axis=1)

    # Preprocesar la variable objetivo utilizando LabelEncoder
    le = LabelEncoder()
    y = df_union['clasificacion']
    y = le.fit_transform(y)  # Codificar categorías a números

    # Definir las variables predictoras (X)
    X = df_union[["distrito_cod", "año", "mes", tipo_delito]]

    # Dividir los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entrenar el modelo de Red Neuronal (MLP)
    modelo = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
    modelo.fit(X_train, y_train)

    # Hacer predicciones en todo el conjunto de datos
    df_union["prediccion"] = modelo.predict(X)

    # Invertir la transformación del LabelEncoder para obtener las etiquetas originales
    df_union["prediccion"] = le.inverse_transform(df_union["prediccion"])

    # Graficar el mapa
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    df_union.plot(column="prediccion", cmap="tab10", legend=True, ax=ax, edgecolor="black")

    # Agregar nombres de los distritos en el mapa
    for x, y, label in zip(df_union.geometry.centroid.x, df_union.geometry.centroid.y, df_union["DISTRITOS"]):
        ax.text(x, y, label, fontsize=8, ha="center", color="black")

    ax.set_title(f"Mapa de clasificación con redes neuronales (MLP) para {tipo_delito}", fontsize=14)
    ax.axis("off")  # Ocultar ejes

    plt.show()

# Crear el widget interactivo para seleccionar el tipo de delito
interact(clasificar_delito, tipo_delito=widgets.Dropdown(options=delitos, description="Delito:"));


Por último, realizamos una clasifiación mediante SVM. Al igual que en el caso anterior, los resultados obtenidos han sido muy diferentes respecto a los de los otros algoritmos.

In [ ]:
# Definir la función de clasificación
def clasificar_delito(tipo_delito):

    # Calcular percentiles para el tipo de delito seleccionado
    percentil_bajo = np.percentile(df_union[tipo_delito], 33)  # 33% de los valores más bajos
    percentil_alto = np.percentile(df_union[tipo_delito], 66)  # 33% de los valores más altos

    # Clasificar los distritos en base a los percentiles de delitos
    def clasificar_distrito(row):
        if row[tipo_delito] > percentil_alto:
            return 'Alto riesgo'
        elif row[tipo_delito] > percentil_bajo:
            return 'Riesgo medio'
        else:
            return 'Bajo riesgo'

    # Crear la variable objetivo clasificando los distritos
    df_union['clasificacion'] = df_union.apply(clasificar_distrito, axis=1)

    # Preprocesar la variable objetivo utilizando LabelEncoder
    le = LabelEncoder()
    y = df_union['clasificacion']
    y = le.fit_transform(y)  # Codificar categorías a números

    # Definir las variables predictoras (X)
    X = df_union[["distrito_cod", "año", "mes", tipo_delito]]

    # Dividir los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entrenar el modelo de SVM
    modelo = SVC(kernel='rbf', C=1, gamma='scale', random_state=42)
    modelo.fit(X_train, y_train)

    # Hacer predicciones en todo el conjunto de datos
    df_union["prediccion"] = modelo.predict(X)

    # Invertir la transformación del LabelEncoder para obtener las etiquetas originales
    df_union["prediccion"] = le.inverse_transform(df_union["prediccion"])

    # Graficar el mapa
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    df_union.plot(column="prediccion", cmap="tab10", legend=True, ax=ax, edgecolor="black")

    # Agregar nombres de los distritos en el mapa
    for x, y, label in zip(df_union.geometry.centroid.x, df_union.geometry.centroid.y, df_union["DISTRITOS"]):
        ax.text(x, y, label, fontsize=8, ha="center", color="black")

    ax.set_title(f"Mapa de clasificación con SVM para {tipo_delito}", fontsize=14)
    ax.axis("off")  # Ocultar ejes

    plt.show()


# Crear el widget interactivo para seleccionar el tipo de delito
interact(clasificar_delito, tipo_delito=widgets.Dropdown(options=delitos, description="Delito:"));


Como se puede observar, el tipo de algoritmo ha hecho que los resultados varíen mucho. Esto solo es una prueba para realizar clasificaciones y si quisieramos realizar una clasificación mejor deberíamos modificar parámmetros de los algoritmos como el número de neuronas de la red neuronal, C (Parámetro de penalización del error en SVM) o la profundidad del árbol en los árboles de decisión y en random forest. Además, para poder utilizar estas clasificaciones en otros estudios deberíamos validarlas con otros datos, ya que aunque los algoritmos utilizados utilizan una parte de los datos proporcionados para validar, al ser datos extraídos de la misma fuente pueden estar sesgados.

Por ello, no sacaremos ninguna conclusión de los resultados obtenidos a partir de estas clasificaciones porque no tenemos la certeza de que algunos de ellos sea correcto.

**Conclusiones extraídas de los resultados**

Como se ha podido observar en este ESDA, la mayoría de los delitos, independientemente de la fecha, no presentan autocorrelación global estadísticamente significativa. En cuanto a la autocorrelación local, si que está presente y es muy diferente según el tipo de delito y la fecha que estemos estudiando.

Por otra parte, si nos fijamos en la agrupación de zonas según los delitos y la comparamos con las divisiones administrativas reales, podemos ver que son muy diferentes entre sí. Este resultado podría ser útil de cara a luchar contra ciertos crímenes o para la distribución de las fuerzas de seguridad en la ciudad de Madrid.

Por último, del análisis de patrones de puntos realizado sobre los datos de multas en Madrid podemos extraer que las multas se pueden agrupar en clústeres, lo que podría estar indicando que hay zonas más problemáticas en este sentido.

## **5. Evaluación crítica**

Antes de tomar los resultados obtenidos en este análisis exploratorio de datos como definitivos es importante tener en cuenta las debilidades y fortalezas del mismo.

Comenzando por las debilidades, al igual que sucedía en el EDA realizado previamente y aunque se han incluido más fuentes de datos, sigue siendo interesante la inclusión de otras variables que podrían ser interesantes en este estudio como la renta media por persona, el nivel de estudios medios por persona o porcentaje de personas en el paro entre otros muchos ejemplos. Otra posible debilidad es que, aunque hemos utilizado dos tipos de datos (datos puntuales y distritos de Madrid), quizá podría ser interesante hacer este mismo análisis a otras escalas. Por ejemplo a nivel de barrio o incluso de calle, lo que nos permitiría darle otro enfoque al trabajo y comprender mejor la distribución de los delitos en la ciudad de Madrid.  Otra posible debilidad es que no se han comparado los resultados obtenidos con los de otro ESDA similar, ya que no hemos podido encontrar ninguno parecido. Esto nos habría permitido "validar" nuestros resultados, encontrar fallos o fuentes de error y mejorar nuestro ESDA. En cuanto a las estadísticas relativas a las multas de tráfico, la limpieza de los datos no fue sencilla, ya que habían coordenadas vacías y también coordenadas incorrectas. Además, muchas de las coordenadas estaban repetidas, es decir, en la misma coordenada hay datos de varias multas, lo cual puede deberse a la propia naturaleza del hecho o una incorrecta transposición de las coordenadas al crear la base de datos.


En cuanto a las fortalezas, los datos de partida son fiables ya que se han extraido de instituciones oficiales como el Ayuntamiento de Madrid. Además, existe continuidad en los datos, lo que nos permite estudiar la evolución de los delitos sin riesgo a que la falta de datos sesgue nuestros resultados. Otra fortaleza del estudio es que como los datos están divididos en función de distritos, nos permiten ver diferencias dentro de la ciudad de Madrid y dan pie a realizar un analisis espacial de los datos en el futuro. También es una fortaleza que los datos cubran todos los distritos de la ciudad, ya que esto significa que la cobertura espacial de la zona objetivo de estudio es total. Por último, una gran fortaleza del estudio es que se han podido responder, en mayor o menor medida, las preguntas que nos hemos planteado al inicio del estudio, permitiendonos cumplir nuestros objetivos iniciales.